In [1]:
# set os at top 
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
os.environ['HF_HOME'] = '/data1/wln/hf_cache'

import torch
from transformers import AutoProcessor, Blip2ForConditionalGeneration, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# load model and processor
processor = AutoProcessor.from_pretrained('Salesforce/blip2-opt-2.7b')
base_model = Blip2ForConditionalGeneration.from_pretrained(
    'Salesforce/blip2-opt-2.7b', 
    local_files_only=True,
    quantization_config=bnb_config
)

base_model = prepare_model_for_kbit_training(base_model)

# set training args 
training_args = TrainingArguments(
    output_dir= 'lora_adapter_loss_x_0312xx',
    num_train_epochs=2,
    per_device_train_batch_size=4,
)

# lora config
lora_config = LoraConfig(
    r=16, #8
    lora_alpha=32, #16 
    lora_dropout=0.05, #0.05
    bias="none"
)

# get model for training
adapter_model = get_peft_model(base_model, lora_config)

/data1/wln/miniconda3/envs/llm-pbe/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`low_cpu_mem_usage` was None, now default to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.20s/it]


In [2]:
from datasets import load_from_disk
from torch.utils.data import DataLoader, Dataset
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class ImageCaptionDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor
    
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        item = self.dataset[index]
        encoding = self.processor(
            images = item['image'],
            return_tensors="pt"
        )
        for k,v in encoding.items():  encoding[k] = v.squeeze()
        encoding['caption'] = item['caption'][0] 

        return encoding

def collate_fn(batch):
    processed_batch = {}
    for key in batch[0].keys():
        if key != 'caption':
            processed_batch[key] = torch.stack([item[key] for item in batch])
        else:
            tokenized_caption = processor.tokenizer(
                [item[key] for item in batch],
                padding=True,
                return_tensors="pt"
            )
            processed_batch['input_ids'] = tokenized_caption['input_ids']
    
    return processed_batch



# load train set
train_ds = load_from_disk('../dataset/train_dataset')

# convert a huggingface dataset type to pytorch dataset type
train_ds_pt = ImageCaptionDataset(train_ds, processor=processor)
train_dataloader = DataLoader(train_ds_pt, shuffle=True, batch_size=training_args.per_device_train_batch_size, collate_fn=collate_fn)

In [3]:
from transformers import get_scheduler
from tqdm import tqdm

# Initialize optimizer and learning rate scheduler
optimizer = torch.optim.AdamW(adapter_model.parameters(), lr=1e-5, eps=1e-5)
num_training_steps = len(train_dataloader) * 2  # Assume 2 epochs
num_warmup_steps = int(num_training_steps * 0.1)  # 10% warmup

lr_scheduler = get_scheduler(
    "cosine",
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps,
)

adapter_model.to(device)
adapter_model.train()

epoch_loss = 0.0
round_loss = 0.0
for epoch in range(training_args.num_train_epochs):
    print("Epoch:", epoch)
    for step, batch in enumerate(tqdm(train_dataloader)):
        input_ids = batch['input_ids'].to(device)
        pixel_values = batch['pixel_values'].to(device, torch.float16)
        
        outputs = adapter_model(
            input_ids = input_ids,
            pixel_values = pixel_values, 
            labels = input_ids
        )
        
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        optimizer.zero_grad()
        torch.cuda.empty_cache()
        
        epoch_loss += loss.item()
        round_loss += loss.item()
        if (step+1) % 10 == 0: 
            # Extract logits from the model output
            logits = outputs.logits  # Shape: (batch_size, seq_len, vocab_size)
            # Get predicted token IDs by taking the argmax over the vocab dimension
            predicted_ids = torch.argmax(logits, dim=-1)  # Shape: (batch_size, seq_len)
            # Limit the number of tokens to 30 before decoding
            seq_length = 32
            predicted_ids = predicted_ids[:, :seq_length]  # Truncate to max 30 tokens
            # Convert token IDs to text using tokenizer
            decoded_output = processor.tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)
            # Print generated text
            print("Generated Output:", decoded_output)

            # Print avg loss for 10 training samples
            print(round_loss/10)
            round_loss = 0.0

    
    print(f"Epoch {epoch} - Avg Loss: {epoch_loss / len(train_dataloader)}")
    epoch_loss = 0.0


Epoch: 0


  0%|          | 0/1759 [00:00<?, ?it/s]/data1/wln/miniconda3/envs/llm-pbe/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/data1/wln/miniconda3/envs/llm-pbe/lib/python3.10/site-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/data1/wln/miniconda3/envs/llm-pbe/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.au

Generated Output: ['I lot who her wheelchair dress and a with a man in a blue shirt. walking through a street.\n:\n\n\n', 'I lot who his wheelchair shirt-shirt and a picture of front middle of the street. a people toilets in the background.\n', 'I lot of up standard in the ground, the on a chaired platformbike. on to a white in cover. The is', 'I lot is a a shirt shirt. jersey a around the field. grass grass. a chased by a man.\n\n覚醒']
4.952086782455444


  1%|          | 20/1759 [00:47<17:35,  1.65it/s]

Generated Output: ['I only version the most hair is the guy of the guy with the blue shirt.\n.\n\n\n', 'Iovoon are women peopleial the v hats were in a hill in the water, spacesista of\n', 'I lot who in a mountain net in the beach in a men watch from She\ntnc%dfx enthusi antioxid', 'I lot who his jeans and a boots boots was standing the paintetti off the wall in with red.\n']
5.034932327270508


...

Generated Output: ['A the river street, a man is a blue shirt andassemb of a cigarette bag. trash. a dump trash green dump metal can.', 'A man boy in a red shirt and down a hill slide.', 'Aick of people are the dog are a are a hat bag are her head are sitting a people. in a tree.', 'A men large are in the woods of a road. a but a few radio and and a tent. a on']
2.4017796397209166


 99%|█████████▉| 1750/1759 [31:59<00:05,  1.50it/s]

Generated Output: ['A man in a blue and aringsakes is a bags bags. a street street. a few of a womanang man in', 'A man is on a st chair with a. a spoonib on.', 'A man boy is a playingiring aets on a puppet show. a end area center.', 'A manman is in a back to the camera, front of a fire of with fire.']
2.2245187520980836


100%|██████████| 1759/1759 [32:05<00:00,  1.09s/it]

Epoch 1 - Avg Loss: 2.1967514446338785


In [4]:
from peft import PeftModel

# Assuming your model is wrapped with LoRA
adapter_model.save_pretrained("lora_adapter_loss_2_03131235")


In [ ]:
train_ds.save_to_disk("../dataset/train_dataset")
val_ds.save_to_disk("../dataset/val_dataset")
test_ds.save_to_disk("../dataset/test_dataset")
